### evaluating the output forecastings

based on: Evaluate-forecasts-paper-v5.ipynb

output of forecastings format:

    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign	endlap	pred_endlap
      11	12	31	3.0	5.0	2.0	1	1.0	-2.0	-1	49	58
    
refer to:
    19.RankNet/stage_model_regressor.ipynb
    19.RankNet/RankForecasting-stint-paper-1kpitmodel.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
#import indycar.model.stint_simulator_shortterm_pitmodel as stint
#import indycar.model.stint_simulator_paper as stint

import indycar.model.quicktest_simulator as stint

# import all functions 
#from indycar.model.global_variables import _hi
import indycar.model.global_variables as gvar
from indycar.model.quicktest_modules import *

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def load_dfout(datafile, basedir = '../result/22.PaperFinal/'):
    datafile = basedir + datafile  
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout[0]
    
def load_dfout_all(datafile):
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    datafile = '../result/22.PaperFinal/' + datafile  
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout

## 1. evalute stint restuls

In [81]:
events_info = {
    'Phoenix':(256, 1.022, 250),'Indy500':(500,2.5,200),'Texas':(372,1.5,248), 
    'Iowa':(268,0.894,300),'Pocono':(500,2.5,200),'Gateway':(310,1.25,248)
}

years = ['2013','2014','2015','2016','2017','2018','2019']
events = [f'Indy500-{x}' for x in years]

events.extend(['Phoenix-2018','Texas-2018','Texas-2019','Pocono-2018','Pocono-2019','Iowa-2018','Iowa-2019',
              'Gateway-2018','Gateway-2019'])

events_id={key:idx for idx, key in enumerate(events)}

dataroot = 'test/'

_train_events = [events_id[x] for x in [f'Indy500-{x}' for x in ['2013','2014','2015','2016','2017']]]
_test_event = 'Gateway-2018'
_test_events = ['Indy500-2018', 'Indy500-2019', 'Phoenix-2018','Texas-2018','Texas-2019','Pocono-2018','Pocono-2019','Iowa-2018','Iowa-2019',
              'Gateway-2018','Gateway-2019']

In [56]:
gvar.events = events
gvar.events_id  = events_id
gvar.events_info = events_info


gvar.maxlap = get_event_info(_test_event)[2]
#gvar.LAPTIME_DATASET = LAPTIME_DATASET

In [57]:
### load ml models
_trim = 0
plen = 2
_context_len = 60

_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
#preddf = load_dfout(outfile)
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-oracle-t{_trim}.pickle'
#preddf_oracle = load_dfout(outfile)
version = f'IndyCar-d{len(events)}'
outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'

In [23]:
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}-newtry-tuned.pickle'
#preddf = load_dfout(outfile)
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-oracle-t{_trim}-newtry-tuned.pickle'
#preddf_oracle = load_dfout(outfile)

## 2. Evaluation shortterm forecasting results

In [82]:
#
# Model,SignAcc,MAE,50-Risk,90-Risk
# 
cols = ['Year','Model','Top1Acc','SignAcc','MAE','50-Risk','90-Risk']
models = {'currank':'CurRank','rf':'RandomForest','svr':'SVM','xgb':'XGBoost'}
#plen=2
#usemeanstr='median'
usemeanstr='mean'
stint_result = {}

_context_len = 60

In [86]:
_test_events = ['Indy500-2018', 'Indy500-2019']

In [87]:
for plen in [2]:
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')

    retdata = []
    #for year in ['Indy500-2018','Indy500-2019','Phoenix-2018']:
    #for year in [_test_event]:
    for year in _test_events:
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[2]
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            #dfout, accret = eval_sync(preddf[year][clf],errlist[year])
            dfout = shortterm_df[year][clf]
            accret = stint.get_evalret_shortterm(dfout)[0]

            fsamples, ftss = df2samples(dfout)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)

            retdata.append([year,models[clf],accret[0], accret[4], accret[1], prisk_vals[1], prisk_vals[2]])
            
        if False:
            #oracle
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-oracle-l10-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret, pret = _all[0],_all[1],_all[2],_all[3]
            allsamples, alltss = get_allsamples(year=year, model='oracle')
            #_, pret[mid]= prisk_direct_bysamples(ret[mid][0][1], ret[mid][0][2])
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)

            dfout = do_rerank(ranknetdf[year]['oracle_mean'])
            accret = stint.get_evalret_shortterm(dfout)[0]
            #fsamples, ftss = runs2samples_ex(ranknet_ret[f'oracle-RANK-{year}-inlap-nopitage'],[])
            #_, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            retdata.append([year,'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #pitmodel
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret = _all[0],_all[1],_all[2]

            dfout = do_rerank(ranknetdf[year]['pitmodel_mean'])
            accret = stint.get_evalret_shortterm(dfout)[0]
            #fsamples, ftss = runs2samples_ex(ranknet_ret[f'pitmodel-RANK-{year}-inlap-nopitage'],[])
            #_, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            allsamples, alltss = get_allsamples(year=year, model='pitmodel')
            #_, pret[mid]= prisk_direct_bysamples(ret[mid][0][1], ret[mid][0][2])
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)        
            retdata.append([year,'RankNet-MLP',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #print('oracle:', ranknet_pret[f'oracle-RANK-2018-inlap-nopitage'])
            #print('pitmodel:', ranknet_pret[f'pitmodel-RANK-2018-inlap-nopitage'])

    stint_result[plen] = pd.DataFrame(data=retdata, columns=cols)
    
    print(stint_result[plen])

year: Indy500-2018 clf: currank
testset size: 5340 minlap: 9 maxlap: 196
model: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}, top1: {188}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.09733723382543032, 0.0973263487312062, 0.0973154636369821])
year: Indy500-2018 clf: rf
testset size: 5340 minlap: 9.0 maxlap: 196.0
model: acc={0.52}, mae={1.76}, rmse={3.38},r2={0.84}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}, top1: {188}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.12765222123954015, 0.127641336145316, 0.12763045105109186])
year: Indy500-2018 clf: svr
testset size: 5340 minlap: 9.0 maxlap: 196.0
model: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}, top1: {188}


...

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.09076943874353269, 0.0907694387435327, 0.09076943874353272])
year: Indy500-2019 clf: svr
testset size: 5629 minlap: 9.0 maxlap: 196.0
model: acc={0.73}, mae={1.16}, rmse={3.05},r2={0.88}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}, top1: {188}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.080397193103194, 0.08039719310319401, 0.08039719310319403])
year: Indy500-2019 clf: xgb
testset size: 5629 minlap: 9.0 maxlap: 196.0
model: acc={0.64}, mae={1.25}, rmse={2.88},r2={0.89}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.73}, mae={1.18}, rmse={3.07},r2={0.88}, top1: {188}
dict_values([0.08622236423295768, 0.08622236423295769, 0.08622236423295769])
           Year         Model   Top1Acc   SignAcc       MAE   50-Risk  \
0  Indy500-2018       CurRank  0.718085  0.594944  1.339513  0.097326   
1  Indy500-2018  RandomForest  0.515957  0.564981  1.761610  0.127641   
2  Indy500-2018           SVM  0.718085  0.594944  1.339513  0.097326   
3  Indy500-2018       XGBoost  0.462766  0.626030  1.630899  0.118471   
4  Indy500-2019       CurRank  0.734043  0.643631  1.162551  0.080397   
5  Indy500-2019  RandomForest  0.606383  0.642388  1.313910  0.090769   
6  Indy500-2019           SVM  0.734043  0.643631  1.162551  0.080397   
7  Indy500-2019       X

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


In [85]:
stint_result[2]

,Year,Model,Top1Acc,SignAcc,MAE,50-Risk,90-Risk
0,Phoenix-2018,CurRank,0.903361,0.834917,0.409343,0.040474,0.040443
1,Phoenix-2018,RandomForest,0.504202,0.403207,1.766429,0.174110,0.174079
2,Phoenix-2018,SVM,0.903361,0.835313,0.408947,0.040435,0.040404
3,Phoenix-2018,XGBoost,0.155462,0.267023,3.458828,0.341645,0.341613
4,Texas-2018,CurRank,0.932203,0.790640,0.446485,0.049508,0.049508
5,Texas-2018,RandomForest,0.470339,0.348634,2.195701,0.242825,0.242825
6,Texas-2018,SVM,0.932203,0.790640,0.446485,0.049508,0.049508
7,Texas-2018,XGBoost,0.148305,0.268473,3.947604,0.437581,0.437581
8,Texas-2019,CurRank,0.898305,0.823818,0.356625,0.037584,0.037584
9,Texas-2019,RandomForest,0.207627,0.361099,1.971879,0.207499,0.207499


In [63]:
dfs = []
for plen in stint_result:
    df = stint_result[plen]
    df['plen'] = plen
    
    dfs.append(df)
    
alldf = pd.concat(dfs)
    

In [64]:
alldf

,Year,Model,SignAcc,MAE,50-Risk,90-Risk,plen
0,Phoenix-2018,CurRank,0.903361,0.409343,0.040474,0.040443,2
1,Phoenix-2018,RandomForest,0.504202,1.766429,0.174110,0.174079,2
2,Phoenix-2018,SVM,0.903361,0.408947,0.040435,0.040404,2
3,Phoenix-2018,XGBoost,0.155462,3.458828,0.341645,0.341613,2
4,Texas-2018,CurRank,0.932203,0.446485,0.049508,0.049508,2
5,Texas-2018,RandomForest,0.470339,2.195701,0.242825,0.242825,2
6,Texas-2018,SVM,0.932203,0.446485,0.049508,0.049508,2
7,Texas-2018,XGBoost,0.148305,3.947604,0.437581,0.437581,2
8,Texas-2019,CurRank,0.898305,0.356625,0.037584,0.037584,2
9,Texas-2019,RandomForest,0.207627,1.971879,0.207499,0.207499,2


In [65]:
alldf.to_csv(f'{dataroot}/evaluation_result_shortterm_plen2-8_all_v4-c{_context_len}.csv')

## 3. Split by pit-covered-laps and normal laps

define pit covered laps := two laps before and after a pit

split shortterm-results only

In [66]:
# get pit laps, pit-covered-laps
# pitdata[year] = [pitlaps, pitcoveredlaps]
covergap = 1
with open(f'{dataroot}/pitcoveredlaps-alldata-g{covergap}.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    pitdata = pickle.load(f, encoding='latin1') 

In [67]:
def clear_samples(full_samples, full_tss, clearidx):
    """
    clear the laps in clearidx
    """
    
    carlist = full_tss.keys()
    
    for carid, carno in enumerate(carlist):
        forecast = full_samples[carno]
        target = full_tss[carno]
        
        forecast[:, clearidx] = np.nan
        target[clearidx] = np.nan
        
        full_samples[carno] = forecast
        full_tss[carno] = target
        
    return full_samples, full_tss

In [88]:
#
# Model,SignAcc,MAE,50-Risk,90-Risk
# 
cols = ['Year','Model','ExpID', 'LapType', 'Top1Acc','SignAcc','MAE','50-Risk','90-Risk']
models = {'currank':'CurRank','rf':'RandomForest','svr':'SVM','xgb':'XGBoost'}

#plen=2
#usemeanstr='median'
usemeanstr='mean'
#stint_split_result = {}

In [90]:
retdata = []
_test_events = ['Indy500-2018', 'Indy500-2019', 'Phoenix-2018','Texas-2018','Texas-2019','Pocono-2018','Pocono-2019','Iowa-2018','Iowa-2019',
              'Gateway-2018','Gateway-2019']
    
for plen in [2]:
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')

    #for year in ['Indy500-2018','Indy500-2019','Phoenix-2018']:
    #for year in [_test_event]:
    for year in _test_events:
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[2]
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            #dfout, accret = eval_sync(preddf[year][clf],errlist[year])
            dfout = shortterm_df[year][clf]
            accret = stint.get_evalret_shortterm(dfout)[0]

            fsamples, ftss = df2samples(dfout)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)

            retdata.append([year,models[clf],'MLMODELS','all', accret[0], accret[4], accret[1], prisk_vals[1], prisk_vals[2]])
            
for laptype in ['normal','pit']:
    
    plen = 2
    
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')


    #for year in ['2018','2019']:
    #for year in [_test_event]:
    for year in _test_events:
        
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[2]
        
        # select the set
        pitcoveredlaps = pitdata[year][1]
        normallaps = set([x for x in range(1,201)]) - pitcoveredlaps
        
        if laptype == 'normal':
            sellaps = normallaps
            clearlaps = pitcoveredlaps
        else:
            sellaps = pitcoveredlaps
            clearlaps = normallaps
        
        
        # pitcoveredlaps start idx = 1
        startlaps = [x-plen-1 for x in sellaps]
        #sellapidx = np.array([x-1 for x in sellaps])
        clearidx = np.array([x-1 for x in clearlaps])
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            
            dfout = shortterm_df[year][clf]
            fsamples, ftss = df2samples(dfout)
            
            fsamples, ftss = clear_samples(fsamples, ftss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            
            # split dfout by startlap
            # curlap = int(dfrec.startlap.values[0] + prediction_len)
            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,models[clf],'MLMODELS',laptype, accret[0], accret[4], accret[1], prisk_vals[1], prisk_vals[2]])
            
        if False:
            #oracle
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-oracle-l10-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret, pret = _all[0],_all[1],_all[2],_all[3]

            dfout = do_rerank(ranknetdf[year]['oracle_mean'])

            allsamples, alltss = get_allsamples(year=year, model='oracle')

            allsamples, alltss = clear_samples(allsamples, alltss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)

            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #pitmodel
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret = _all[0],_all[1],_all[2]

            dfout = do_rerank(ranknetdf[year]['pitmodel_mean'])

            allsamples, alltss = get_allsamples(year=year, model='pitmodel')
            allsamples, alltss = clear_samples(allsamples, alltss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)        

            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,'RankNet-MLP',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #print('oracle:', ranknet_pret[f'oracle-RANK-2018-inlap-nopitage'])
            #print('pitmodel:', ranknet_pret[f'pitmodel-RANK-2018-inlap-nopitage'])

    #stint_split_result[laptype] = pd.DataFrame(data=retdata, columns=cols)
    
    #print(stint_split_result[laptype])
retdf = pd.DataFrame(data=retdata, columns=cols) 
retdf.to_csv(f'{dataroot}/evaluation_result_shortterm_plen2-split-g{covergap}_v5-c{_context_len}.csv')

year: Indy500-2018 clf: currank
testset size: 5340 minlap: 9 maxlap: 196
model: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}, top1: {188}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.09733723382543032, 0.0973263487312062, 0.0973154636369821])
year: Indy500-2018 clf: rf
testset size: 5340 minlap: 9.0 maxlap: 196.0
model: acc={0.52}, mae={1.76}, rmse={3.38},r2={0.84}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}, top1: {188}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.12765222123954015, 0.127641336145316, 0.12763045105109186])
year: Indy500-2018 clf: svr
testset size: 5340 minlap: 9.0 maxlap: 196.0
model: acc={0.72}, mae={1.34}, rmse={3.25},r2={0.85}, top1_pred: {188}, top1_naive: {188}
            naive: acc={0.72}, mae={1.36}, rmse={3.25},r2={0.85}, top1: {188}


...

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.3232743669788415, 0.3232743669788415, 0.32327436697884154])
testset size: 1870 minlap: 40.0 maxlap: 226.0
model: acc={0.34}, mae={3.10}, rmse={4.42},r2={0.43}, top1_pred: {93}, top1_naive: {93}
            naive: acc={0.81}, mae={1.06}, rmse={2.28},r2={0.85}, top1: {93}
year: Gateway-2019 clf: svr


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.08766909469302811, 0.0876690946930281, 0.08766909469302808])
testset size: 1870 minlap: 40.0 maxlap: 226.0
model: acc={0.81}, mae={1.04}, rmse={2.26},r2={0.85}, top1_pred: {93}, top1_naive: {93}
            naive: acc={0.81}, mae={1.06}, rmse={2.28},r2={0.85}, top1: {93}
year: Gateway-2019 clf: xgb
dict_values([0.490894901144641, 0.490894901144641, 0.490894901144641])
testset size: 1870 minlap: 40.0 maxlap: 226.0
model: acc={0.25}, mae={4.91}, rmse={6.64},r2={-0.28}, top1_pred: {93}, top1_naive: {93}
            naive: acc={0.81}, mae={1.06}, rmse={2.28},r2={0.85}, top1: {93}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1703: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


In [91]:
retdf

,Year,Model,ExpID,LapType,Top1Acc,SignAcc,MAE,50-Risk,90-Risk
0,Indy500-2018,CurRank,MLMODELS,all,0.718085,0.594944,1.339513,0.097326,0.097315
1,Indy500-2018,RandomForest,MLMODELS,all,0.515957,0.564981,1.761610,0.127641,0.127630
2,Indy500-2018,SVM,MLMODELS,all,0.718085,0.594944,1.339513,0.097326,0.097315
3,Indy500-2018,XGBoost,MLMODELS,all,0.462766,0.626030,1.630899,0.118471,0.118460
4,Indy500-2019,CurRank,MLMODELS,all,0.734043,0.643631,1.162551,0.080397,0.080397
...,...,...,...,...,...,...,...,...,...
127,Gateway-2018,XGBoost,MLMODELS,pit,0.159574,0.244212,5.656009,0.577224,0.577145
128,Gateway-2019,CurRank,MLMODELS,pit,0.806452,0.608556,1.036364,0.087669,0.087669
129,Gateway-2019,RandomForest,MLMODELS,pit,0.344086,0.377540,3.104813,0.323274,0.323274
130,Gateway-2019,SVM,MLMODELS,pit,0.806452,0.608556,1.036364,0.087669,0.087669


In [70]:
stint_split_result['normal']

,Year,Model,SignAcc,MAE,50-Risk,90-Risk
0,Phoenix-2018,CurRank,0.985507,0.027107,0.003654,0.003654
1,Phoenix-2018,RandomForest,0.608696,0.994380,0.120916,0.120916
2,Phoenix-2018,SVM,0.985507,0.027107,0.003654,0.003654
3,Phoenix-2018,XGBoost,0.210145,2.660496,0.290974,0.290974
4,Texas-2018,CurRank,1.000000,0.103935,0.010243,0.010243
5,Texas-2018,RandomForest,0.420000,1.884965,0.197030,0.197030
6,Texas-2018,SVM,1.000000,0.103935,0.010243,0.010243
7,Texas-2018,XGBoost,0.280000,3.474268,0.357931,0.357931
8,Texas-2019,CurRank,0.949580,0.041229,0.005668,0.005668
9,Texas-2019,RandomForest,0.235294,1.396928,0.182283,0.182283


In [71]:
stint_split_result['pit']

,Year,Model,SignAcc,MAE,50-Risk,90-Risk
0,Phoenix-2018,CurRank,0.728571,1.330124,0.101012,0.100931
1,Phoenix-2018,RandomForest,0.342857,3.068776,0.300900,0.300819
2,Phoenix-2018,SVM,0.728571,1.328748,0.100910,0.100829
3,Phoenix-2018,XGBoost,0.114286,4.667125,0.479426,0.479345
4,Texas-2018,CurRank,0.851852,0.859913,0.082026,0.082026
5,Texas-2018,RandomForest,0.444444,2.689287,0.278466,0.278466
6,Texas-2018,SVM,0.851852,0.859913,0.082026,0.082026
7,Texas-2018,XGBoost,0.064815,4.816287,0.492889,0.492889
8,Texas-2019,CurRank,0.785714,0.911285,0.077718,0.077718
9,Texas-2019,RandomForest,0.250000,2.394689,0.286745,0.286745


In [72]:
dfs = []
for laptype in ['normal','pit']:
    df = stint_split_result[laptype]
    df['laptype'] = laptype
    
    dfs.append(df)
    
alldf = pd.concat(dfs)

In [73]:
alldf.to_csv(f'{dataroot}/evaluation_result_shortterm_plen2-split-g{covergap}_v4-c{_context_len}.csv')

In [ ]:
alldf